In [2]:
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import models
import training
from torchvision.transforms import v2
import numpy as np
import matplotlib.pyplot as plts
from os.path import join
import torchvision
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.utils.data import random_split

device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def format_data(x):
    mean = [x[:, n, :, :].mean() for n in range(x.shape[1])]
    std = [x[:, n, :, :].std() for n in range(x.shape[1])]

    X = v2.Compose([
            torch.from_numpy,
            v2.Normalize(mean=mean, std=std),
            v2.Resize((64, 64)),
        ])(x).to(device)
    return X

Using cuda device


In [7]:
orders = range(4,5)
photocounts = [2**i for i in range(6, 12)]

for order in orders:
    for n in photocounts:
        with h5py.File('../../Data/Training/pure_photocount.h5', 'r') as f:
            images = format_data(f[f'images_order{order}/{n}_photocounts'][:])
            labels = torch.from_numpy(f[f'labels_order{order}/{n}_photocounts'][:]).to(device)

        dset = TensorDataset(images,labels)

        train_size = int(0.85 * len(dset))
        test_size = len(dset) - train_size

        train_dataset, test_dataset = random_split(dset, [train_size, test_size])
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=256)

        L = images.shape[2]
        loss_fn = training.fidelity_loss
        n_channels = 2
        n_classes = 2*(order+1)

        model = models.ConvNet(L,L,n_channels, n_classes,[24,40,35],5,nn.ELU,[120,80,40]).to(device)

        optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=.5, min_lr = 1e-5)

        save_path = f"../../Results/MachineLearningModels/Photocount/Pure_Order{order}/{n}_photocounts"

        writer = SummaryWriter(save_path)
        early_stopping = training.EarlyStopping(patience=30,save_path=save_path)
        for t in range(200):
            epoch = t+1
            print(f"-------------------------------\nEpoch {epoch}")
            training.train(model, train_loader, loss_fn, optimizer, device)
            val_loss = training.test(model, test_loader, loss_fn, device, epoch, writer, verbose=True)
            scheduler.step(val_loss)
            early_stopping(val_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break
        print("Done!")
        writer.close()

-------------------------------
Epoch 1
Training loss: 0.36980126
-------------------------------
Epoch 2
Training loss: 0.31751469
-------------------------------
Epoch 3
Training loss: 0.30374380
-------------------------------
Epoch 4
Training loss: 0.28351329
-------------------------------
Epoch 5
Training loss: 0.27638213
-------------------------------
Epoch 6
Training loss: 0.27152120
-------------------------------
Epoch 7
Training loss: 0.27003145
-------------------------------
Epoch 8
Training loss: 0.25839978
-------------------------------
Epoch 9
Training loss: 0.25717710
-------------------------------
Epoch 10
Training loss: 0.25301319
-------------------------------
Epoch 11
Training loss: 0.25062973
-------------------------------
Epoch 12
Training loss: 0.25013929
-------------------------------
Epoch 13
Training loss: 0.25120283
-------------------------------
Epoch 14
Training loss: 0.24762631
-------------------------------
Epoch 15
Training loss: 0.24716482
----

In [8]:
orders = range(1,5)
photocounts = [2**i for i in range(6, 12)]

fids = torch.empty(50,len(orders),len(photocounts))

with h5py.File('../../Data/Processed/pure_photocount.h5', 'r') as f:
    for order in orders:
        labels = torch.from_numpy(f[f'labels_order{order}'][:50]).to(device)
        labels = torch.cat([torch.real(labels), torch.imag(labels)], dim=1)
        for (k,n) in enumerate(photocounts):
            images = format_data(f[f'images_order{order}/{n}_photocounts'][:].astype(np.float32))
            model = torch.load(f"../../Results/MachineLearningModels/Photocount/Pure_Order{order}/{n}_photocounts/checkpoint.pt")
            #model.eval()

            with torch.no_grad():
                fids[:,order-1,k] = 1 - training.fidelity_loss(model(images),labels)

fids = fids.numpy()
fids.mean(axis=0)

array([[0.9638479 , 0.9766399 , 0.9872512 , 0.99163824, 0.9944772 ,
        0.9952853 ],
       [0.92798203, 0.9626988 , 0.97369117, 0.9836429 , 0.9846523 ,
        0.98788923],
       [0.8014133 , 0.8705011 , 0.92280966, 0.9441836 , 0.9686382 ,
        0.9790116 ],
       [0.69152784, 0.77247965, 0.85966736, 0.89261633, 0.92167693,
        0.94233674]], dtype=float32)

In [14]:
with h5py.File('../../Results/Photocount/machine_learning.h5', 'w') as f:
    f.create_dataset('fids', data=fids.mean(axis=0))
    f.create_dataset('fids_std', data=fids.std(axis=0))